<a href="https://colab.research.google.com/github/ignaciomz/Geocodificacion_Masiva/blob/main/Geocodificaci%C3%B3n_Direcciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geocodificación con Pandas y Geopy
**A partir de un archivo Excel con Direcciones particulares, podemos obtener la Latitud y Longitud para georeferenciar en un mapa en PowerBI o cualquier software que trabaje sobre coordenadas**

Lo dificil de este proceso, es que no te dejará geocodificar mas de 12.000 registros promedio por ejecución, dado que estamos usando una API gratuita.
Aquí es importante que se guarde parcialmente un archivo cada n cantidad de registros geocodificados (por si se desconecta la terminal), e ir guardando archivos consecutivos (esto ya un trabajo manual, a tu gusto).

Es un proceso que uso mucho en mi trabajo cotidiano.
Espero sea de ayuda toda la comunidad Data.

In [ ]:
import time
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# Inicializa el geocodificador de Nominatim con un tiempo de espera mayor
geolocator = Nominatim(user_agent="NombredetuAppHipotetica (tumail@gmail.com)", timeout=10)

# Función para geocodificar una dirección con manejo de tiempo de espera y reintentos
def geocode_address(address, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return (location.latitude, location.longitude)
            else:
                return (None, None)
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error: {e}. Intento {attempt + 1} de {retries}.")
            time.sleep(5)  # Espera 5 segundos antes de reintentar
    return (None, None)

# Cargar el archivo de Excel (debe tener una columna solamente con las direcciones)
archivo_excel = 'tuarchivocondirecciones.xlsx'  # Reemplaza con la ruta correcta a tu archivo Excel
df = pd.read_excel(archivo_excel)

# Verifica los nombres de las columnas para identificar la columna de direcciones
print("Columnas en el archivo Excel:", df.columns)

# Define Latitud y Lognitud
df['latitud'] = None
df['longitud'] = None

# Recorre cada dirección y geocodifica con pausas para evitar el bloqueo
guardar_cada_n_filas = 100  # Guardar cada 100 filas
for i, row in df.iterrows():
    direccion = row['Domicilio']  # Aquí puedes definir a tu gusto - Recuerda poner el titulo de la columna según tu Archivo con Direcciones, o Domiciliio, o cualquiera sea la referencia.
    if pd.notna(direccion) and pd.isna(row['latitud']) and pd.isna(row['longitud']):  # Solo procesar si aún no tiene coordenadas
        lat, lon = geocode_address(direccion)
        df.at[i, 'latitud'] = lat
        df.at[i, 'longitud'] = lon
        print(f"Geocodificado: {direccion} -> ({lat}, {lon})")
        time.sleep(1)  # Pausa solo si se procesa exitosamente

        # Guardar el progreso cada n filas
        if i % guardar_cada_n_filas == 0:
            df.to_excel('nombredearchivoguardadotemporal.xlsx', index=False, engine='openpyxl')  # Guardado parcial en ruta y nombre especifico
            print(f"Progreso guardado hasta la fila {i}.")

# Guardar los resultados finales a un nuevo archivo Excel
df.to_excel('nombrearchivofinal.xlsx', index=False, engine='openpyxl')  # Reemplaza con la ruta donde guardar el archivo final
print("Geocodificación completa y guardada.")